In [ ]:
import perceval as pcvl
from collections import Counter
pcvl.__version__

# BasicStates

In Linear Optical Circuits, photons can have many discrete degrees of freedom, called modes. 
It can be the frequency, the polarisation, the position, or all of them.

We represent these degrees of freedom with Fock states. If we have $n$ photons over $m$ modes, the Fock state $|s_1,s_2,...,s_m\rangle$ means we have $s_i$ photons in the $i^{th}$ mode. Note that $\sum_{i=1}^m s_i =n$.

In Perceval, we will use the module `pcvl.BasicState`

## Defining and manipulating BasicStates

In [ ]:
# BasicState can be constructed from a bra-ket string
s=pcvl.BasicState("|0,1>")
# or from a list
assert(s == pcvl.BasicState([0,1]))
print(s[0], s[1])

In [ ]:
print("type of s is", type(s))
# BasicState will print as a string, s.n is the number of photons, s.m the number of modes (which is also the len of the BasicState)
print(f"s='{s}', n={s.n}, m={s.m}, m={len(s)}")

<div class="alert alert-info">
Note the type is <code>exqalibur.FockState</code> and not <code>pcvl.BasicState</code>!<br>
&rarr; <code>perceval</code> library is using behind the scene a c++-optimized library <code>exqalibur</code> that is in charge of all the heavy lifting - we never use it directly, but it is where all the simulation work happens!
</div>

In [ ]:
# BasicState does not support assignment
try:
    s[0]=1
except Exception as e:
    print("ERROR:", e)

In [ ]:
# let us define a function that does that!
def assign(s, k, n_k):
    r"""assigns n_k photon in the k-th mode of s, and returns the new BasicState"""
    assert(isinstance(s, pcvl.BasicState))
    assert(n_k < s.m)
    ### ENTER CODE HERE

    ### END CODE
    return new_s

# check if it works
new_s = assign(s, 0, 1)
assert(new_s == pcvl.BasicState("|1,1>"))

### Working with annotations

Instead of simple fock states, we can all annotate each photon with an annotation. In general an annotated photon is represented by `{x:y}`
where `x` is the name of the annotation, and `y` its value - the value has to be numeric except for polarization.
A special annotation named `P` is the photon polarization and `perceval` knows about its semantic - the value can be `H` (horizontal), `V` (vertical), `D` (diagonal)...

In [ ]:
a_bs = pcvl.BasicState("|{P:H},{P:V},{P:D}>")
print(a_bs, a_bs[0],a_bs[1])

However, you can use any name you like and even combine annotations:

In [ ]:
pcvl.BasicState("|{color:0,P:H},{color:1,P:V}>")

Annotation order does not matter:

In [ ]:
assert(pcvl.BasicState("|{color:0,P:H}>")==pcvl.BasicState("|{P:H,color:0}>"))

You can check if the basic state has an annotation and obtains the annotation of the photons on a specific mode:

In [ ]:
s=pcvl.BasicState("|{P:H,color:0},0,1>")
assert(s.has_annotations)
print(s.get_mode_annotations(0))

Each annotation behaves as a dictionary of key, value:

In [ ]:
photon_idx = 0
for k in range(s.m):
    for annotation in s.get_mode_annotations(k):
        photon_annotations=[]
        for (annot_name,annot_value) in annotation:
            photon_annotations.append(annot_name)
        print(f"photon {photon_idx} in mode {k} has annotations {photon_annotations}")
        photon_idx += 1

In [ ]:
# let us define a function checking if one photon in a basic state has a polarization annotation
def has_polarization(s):
    if not s.has_annotations:
        return False
    ### ENTER CODE HERE

    ### END CODE
    return False
assert(not has_polarization(pcvl.BasicState("|0,1>")))
assert(not has_polarization(pcvl.BasicState("|{color:0}>")))
assert(has_polarization(pcvl.BasicState("|1,{color:0,P:H},{_:1}>")))

Last we can clear all annotations on a basic state:

In [ ]:
print("before:", s)
s.clear_annotations()
print("after:", s)

## Operations on BasicStates

### tensor product
`*` defines tensor product (`s1*s2` ~ $s_1\otimes s_2$)

In [ ]:
print(pcvl.BasicState([0,1])*pcvl.BasicState([2,3]))

And `**` the tensor power (`s**n` ~ $s^{\otimes n}$), typically useful to initialize an input state

In [ ]:
s2 = pcvl.BasicState([0,1])**4
print(s2)

In [ ]:
# python slice operator applies smoothly
print(s2[1:4])

# State Vectors

A state vector is defined as a general quantum state: $\sum_i \alpha_i.s_i$ where $s_i$ are `BasicState`and $\alpha_i$ their complex parameters/

StateVectors can be build directly with `+`, `-` operators can be used to create state superposition:

In [ ]:
sv1 = pcvl.BasicState([0,1])+pcvl.BasicState([1,0])

In [ ]:
# this is again an optimized `exqalibur` object!
type(sv1)

In [ ]:
# let us check the value
print(sv1)

<div class="alert alert-info">So $sv_1=\frac{\sqrt 2}{2}(\ket{1,0}+\ket{0,1})$. Note the <code>sqrt(2)/2</code> that was introduced: what even you are doing on the <code>StateVector</code>, they keep the parameters normalized so that $\sum_i |\alpha_i|^2=1$

In [ ]:
# one easy way to check this is as follows, the multiplication of a StateVector by a constant does not change its value
assert(2*pcvl.StateVector([0,1])==pcvl.StateVector([0,1]))

except when building a superposed state:

In [ ]:
sv2 = pcvl.StateVector([0,1])-2j*pcvl.StateVector([2,3])
print(sv2)

<div class="alert alert-info">
    Did you notice that it is being represented using `2*sqrt(5)/5` ($\frac{2\sqrt{5}}{5}$) and not numerical values? This is a built-in feature trying to recognize remarkable numbers when we print them - this helps the reader. This number simplification is built in the string conversion and can be disabled by addint `nsimplify=False` parameter in the implicit `__str__` function used to print the `StateVector` as follows 
</div>

In [ ]:
# switch off number conversion
print(sv2.__str__(nsimplify=False))

<div class="alert alert-warning">This is clearly less intuitive but we now see that the parameters are actually complex values!!!</div>

We can also use `pdisplay` to propose an even nicer representation, where we see the different components of the `StateVector`:

In [ ]:
pcvl.pdisplay(sv2) # you can add nsimplify=False here too!

Finally, we can iterate manually through the different components of a state vector:

In [ ]:
for bs, alpha in sv2:
    # here we invoke manually the number simplification - easier to read
    print(alpha, pcvl.simple_complex(alpha)[1], bs)

We can combine states with different number of photons (`-2*sqrt(5)*I/5*|2,3>+sqrt(5)/5*|0,1>`) but we cannot combine states with different number of modes - it does not make sense:

In [ ]:
try:
    pcvl.BasicState("|0,1>")+pcvl.BasicState("|0,1,2>")
except:
    print("!!! Exception was generated: we cannot have superposed states with different number of modes !!!")

In [ ]:
# ok - we can now manipulate StateVector - let us define a function giving us the probability of having one or mode photons in a given mode
def photon_probability(sv, mode):
    r"""should return the probability that when we measure `sv`, we observe at least one photon in mode `mode`"""
    assert(mode < sv.m)
    probability = 0
    ### ENTER CODE HERE

    ### END CODE
    return probability

In [ ]:
# let us check this
assert(abs(photon_probability(pcvl.BasicState([0,1])+pcvl.BasicState([1,0]),0)-0.5)<1e-6)
assert(abs(photon_probability(pcvl.BasicState([0,1,0])+pcvl.BasicState([0,1,0]),0)-0)<1e-6)
assert(abs(photon_probability(sv2, 0)-0.8)<1e-6)

Now we can also use built-in `StateVector` methods providing sampling and measurement: 

In [ ]:
print(sv2)
c = Counter()
for s in sv2.samples(100): 
    c[s] += 1
print(c)

In [ ]:
sv_4 = pcvl.StateVector("|0,1,1>")-1j*pcvl.StateVector("|1,1,0>")
map_measure_sv_4 = sv_4.measure([1])
for s, (p, sv) in map_measure_sv_4.items(): 
    print(s, p, sv)

In [ ]:
map_measure_sv_4 = sv_4.measure([2])
for s, (p, sv) in map_measure_sv_4.items(): 
    print(s, p, sv)

## Modeling of a noisy photon source with a SVDistribution

A `SVDistribution` is a mixed state of `StateVector` - it is commonly used to model noisy photon source.

For that let us use the `pcvl.Source` object, it takes the following parameters:
* `emission_probability`: `float` = 1,
* `multiphoton_component`: `float` = 0,
* `indistinguishability`: `float` = 1,
* `losses`: `float` = 0
  

### Perfect Source

<img src="img/perfect_source.png">

In [ ]:
perfect_source = pcvl.Source(emission_probability=1, multiphoton_component=0, indistinguishability=1, losses=0)
pcvl.pdisplay(perfect_source.probability_distribution())

<div class="alert alert-info">
    In the perfect case, the source emits a sequence of `|1>`
</div>

### Real World Photon source

<img src="img/realworld_source.png">

In [ ]:
realworld_source = pcvl.Source(emission_probability=0.5, multiphoton_component=0.02, indistinguishability=0.95, losses=0.7)
pcvl.pdisplay(realworld_source.probability_distribution())

<div class="alert alert-warning">
    In the real world, with the parameters given to the source, 85% of the time, no photon is detected, 14.5% of the time, single indistinguishable photons are emitted (represented with <code>{_:0}</code>) - and the rest is a combination of g_2 and distinguishable photons
</div>

# LO-Components

In [ ]:
import perceval.components.unitary_components as pcvl_comp

In [ ]:
perm=pcvl_comp.PERM([2,0,1])
# name of the component
print(perm.name)
# the definition of the component
print(perm.describe())
# the unitary matrix
pcvl.pdisplay(perm.definition())
# and the visual representation
pcvl.pdisplay(perm)

In [ ]:
bs=pcvl_comp.BS()
# name of the component
print(bs.name)
# the definition of the component
print(bs.describe())
# the unitary matrix
pcvl.pdisplay(bs.definition())
# the actual unitary matrix
pcvl.pdisplay(bs.compute_unitary())
# the visual representation
pcvl.pdisplay(bs)